In [45]:
# !pip install transformers
# !pip install pandas
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# !nvidia-smi

In [34]:
# read xml file
import os
import xml.etree.ElementTree as ET

all_paths = []
for root, dirs, files in os.walk("data/2009"):
    for file in files:
        if file.endswith(".xml"):
             all_paths.append(os.path.join(root, file))

def read_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()

    text = []
    for elem in root.iter():
        if elem.text is not None:
            text.append(elem.text.strip())

    extracted_text = " ".join(text)

    

    # remove prescript
    # extracted_text = extracted_text[1030:].strip()
    prescript_len = len(extracted_text.split('      ')[0])
    extracted_text = extracted_text[prescript_len:].strip()

    return extracted_text

def save(path, data):
    with open(path, 'w') as f:
        f.write(data)


def clean_dir(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            os.remove(os.path.join(root, file))

clean_dir('processed_data')
for path in all_paths:
    data = read_xml(path)
    path = path.replace('data/2009', 'processed_data').replace('.xml', '.txt')
    save(path, data)

In [36]:
# split the dataset
from sklearn.model_selection import train_test_split

all_paths = []
for root, dirs, files in os.walk("processed_data"):
    for file in files:
        if file.endswith(".txt"):
             all_paths.append(os.path.join(root, file))

train, test = train_test_split(all_paths, test_size=0.2, random_state=42)

In [93]:
from tokenizers import ByteLevelBPETokenizer

# Initialize the tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training options
tokenizer.train(files=train, vocab_size=50000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save the tokenizer
if(os.path.isdir("./tokenizer") == False):
    os.mkdir("./tokenizer")
    
tokenizer.save_model("./tokenizer/custom_tokenizer")

# tokenizer.save_model("./tokenizer/custom_tokenizer")

Exception: Not a directory (os error 20)

In [60]:
# from tokenizers import ByteLevelBPETokenizer

# tokenizer = ByteLevelBPETokenizer(
#     vocab_file="./tokenizer/custom_tokenizer/vocab.json",
#     merges_file="./tokenizer/custom_tokenizer/merges.txt",
# )


TypeError: __init__() got an unexpected keyword argument 'vocab_file'

In [112]:
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset


# tokenizer.encode("Hello, y'all! How are you 😁 ?")

def load_dataset(train_path, test_path):
    train_dataset = Dataset.from_text(train_path)
    test_dataset = Dataset.from_text(test_path)

    train_dataset = tokenizer.encode_batch(train_dataset["text"])
    test_dataset = tokenizer.encode_batch(test_dataset["text"])

    # datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    
    return train_dataset, test_dataset, data_collator

def change_transformers_dataset_2_right_format(dataset, label_name): 
    return dataset.map(lambda example: {'label': example[label_name]}, remove_columns=[label_name])


train_dataset, test_dataset, data_collator = load_dataset(train, test)







Resolving data files: 100%|██████████| 149/149 [00:00<00:00, 180830.81it/s]
Found cached dataset text (/Users/haukurbirgisson/.cache/huggingface/datasets/text/default-536aa9e4b1a200a5/0.0.0)





Resolving data files: 100%|██████████| 38/38 [00:00<00:00, 54677.03it/s]
Found cached dataset text (/Users/haukurbirgisson/.cache/huggingface/datasets/text/default-92caa13081f1df0b/0.0.0)


AttributeError: 'list' object has no attribute 'map'

In [110]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

# Initialize a GPT-2 model from scratch
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-isl-parla",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    eval_steps=400,  # Number of update steps between two evaluations
    save_steps=800,  # After how many steps the model is saved
    warmup_steps=500,  # Number of warmup steps for the learning rate scheduler
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /Users/haukurbirgisson/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-

In [111]:
trainer.train()

***** Running training *****
  Num examples = 1
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3






KeyError: 0